In [10]:
import pandas as pd

MESES = ['01','02','03','04','05','06','07','08','09','10','11','12']
ANOS = ['2022','2023']


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [11]:
###Definindo Função para criar link dos aquivos desejados###
def url_mes_ano (mes, ano):
    mes=str(mes)
    ano=str(ano)
    api_url_prefix='https://api.portaltransparencia.sead.caucaia.ce.gov.br/funcionario/excel/1/'
    api_url_final=api_url_prefix + mes + '/' + ano
    return api_url_final


In [12]:
#### criando funcao pra coletar todos os dados ####
def download_all_data(meses:list, anos:list):
    df_geral = pd.DataFrame()
    for ano in anos:
        for mes in meses:
            try:
                df = pd.read_excel(url_mes_ano(mes,ano), header=1) # ler o arquivo diretamente do link gerado
                df = df.dropna(subset=['Nome']) #limpa as linhas que nao possuem nomes
                df['Mes'] = mes #cria uma coluna e coloca o mes
                df_geral = pd.concat([df_geral,df])
                
            except:
                pass
    return df_geral.reset_index() # retorna o DF com os indicis corrigidos


In [14]:
####fazendo o download de tudo e salvando #####
# df = download_all_data(meses=MESES,anos=ANOS)
# df.to_csv('dados.csv')

In [40]:
#### carregando arquivos ###
df = pd.read_csv('dados.csv', index_col='index')
df = df.reset_index()
df = df.drop(columns=['Unnamed: 0','index'])



In [62]:
def converter_texto_em_float(texto:str):
    try:
        texto = texto.replace('R$','').strip()
        texto = texto.replace(',','.')
        return float(texto)
    except:
        return texto
    


In [70]:
df2= df.map(converter_texto_em_float)
df2.dtypes

Matricula      int64
Nome          object
Orgão         object
Setor         object
Cargo         object
Cargo2       float64
Provento     float64
Desconto     float64
Liquido       object
Mes            int64
dtype: object